# Load librarys

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
import cv2
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

# Load Data

In [2]:
# Dataset
import zipfile

z = zipfile.ZipFile('C:/VS_Repos/Computer_vision/archive.zip')

z.extractall()

In [5]:
folder = 'C:/VS_Repos/Computer_vision/classifier_brain_tumor/tumor'
count = 1

for filename in os.listdir(folder):
    source = folder + filename
    destination = folder + "Y_" +str(count)+".jpg"
    os.rename(source, destination)
    count+=1
print("All files are renamed in the tumor dir.")

All files are renamed in the tumor dir.


In [6]:
folder = 'C:/VS_Repos/Computer_vision/classifier_brain_tumor/no_tumor'
count = 1

for filename in os.listdir(folder):
    source = folder + filename
    destination = folder + "N_" +str(count)+".jpg"
    os.rename(source, destination)
    count+=1
print("All files are renamed in the no_tumor dir.")

All files are renamed in the no_tumor dir.


In [7]:
listyes = os.listdir("C:/VS_Repos/Computer_vision/classifier_brain_tumor/tumor")
number_files_yes = len(listyes)
print(number_files_yes)

listno = os.listdir("C:/VS_Repos/Computer_vision/classifier_brain_tumor/no_tumor")
number_files_no = len(listno)
print(number_files_no)

0
0
